# Programmation d'un modèle de langue n-gram

Dans le cadre du projet Stat'App

In [1]:
import numpy as np

## Traitement des données d'apprentissage

On utilise ici l'extrait assez faible (15 Mo) disponible sur le drive de Benjamin.

In [3]:
fichier = open("./Desktop/fr_wikipedia_sample.txt","r",encoding="utf8")
corpus=fichier.read()
fichier.close()

In [4]:
print(corpus[:500])

Paul Jules Antoine Meillet , né le 11 novembre 1866 à Moulins ( Allier ) et mort le 21 septembre 1936 à Châteaumeillant ( Cher ) , est le principal linguiste français des premières décennies du XXe siècle .
Il est aussi philologue .
Biographie D' origine bourbonnaise , fils d' un notaire de Châteaumeillant ( Cher ) , il fait ses études secondaires au lycée de Moulins .
Étudiant à la faculté des lettres de Paris à partir de 1885 où il suit notamment les cours de Louis Havet , il assiste également


In [7]:
#corpus = corpus.replace(",","").replace("-","").replace("'","' ")

In [5]:
#Virer la casse
corpus = corpus.lower()

#Virer la ponctuation et la remplacer par les balises <s> et </s>
corpus = corpus.replace(".","</s> <s>").replace("!","</s> <s>").replace("?","</s> <s>").replace("\n"," ")
corpus = "<s> "+corpus
#Enlever l'espace à la fin
corpus = corpus[:len(corpus)-1]

Interrogation : Faut-il traiter les virgules, parenthèses et autres symboles comme des mots ?

In [15]:
seqcorpus = corpus.split(' ')

In [16]:
seqcorpus[:9]

['<s>', 'paul', 'jules', 'antoine', 'meillet', ',', 'né', 'le', '11']

In [17]:
vocabulaire = list(set(seqcorpus))

In [18]:
print(len(vocabulaire))

113895


## Apprentissage des fréquences des n-grams

In [117]:
#Paramètre n du n-gram
n = 4

Chaines de n-1 mots

In [118]:
#Compte les fréquences de tous les n-1-grams et les stocke dans un dictionnaire
frequenciesnmoinsun = {}

for i in range(0,len(seqcorpus)-n+2):
    nmoinsungram = ' '.join(seqcorpus[i:i+n-1])
    frequenciesnmoinsun[nmoinsungram] = frequenciesnmoinsun.get(nmoinsungram,0)+1
    
totalsize = 1 #len(frequenciesnmoinsun) 

In [119]:
def frequencynmoinsun(text):
    return frequenciesnmoinsun.get(text,0)/totalsize

Chaines de n mots

In [120]:
#Compte les fréquences de tous les n-grams et les stocke dans un dictionnaire
frequenciesn = {}

for i in range(0,len(seqcorpus)-n+1):
    ngram = ' '.join(seqcorpus[i:i+n])
    frequenciesn[ngram] = frequenciesn.get(ngram,0)+1
    
totalsize = 1 #len(frequenciesn)

In [121]:
def frequencyn(text):
    return frequenciesn.get(text,0)/totalsize

In [122]:
#Retourne la probabilité conditionnelle d'un mot sachant le n-1-gram qui le précède
def probacond(mot,seqprec):
    if frequencynmoinsun(seqprec)==0:
        return 0
    return frequencyn(seqprec+" "+mot)/frequencynmoinsun(seqprec)

Test (dans le cas bigram)

In [26]:
print(frequencynmoinsun("constitue"))
print(frequencyn("constitue un"))
print(probacond("un","constitue"))

255.0
37.0
0.1450980392156863


Test (dans le cas trigram)

In [35]:
print(frequencynmoinsun("est un"))
print(frequencyn("est un important"))
print(probacond("important","est un"))

1674.0
3.0
0.0017921146953405018


## Utilisation pour la génération de texte

Greedy Search Method

In [72]:
#Génère une séquence de nbmots mots conditionnellement à la séquence précédente seqprec
def generergreedy(nbmots,seqprec):
    for i in range(nbmots):
        #Construction du vecteur des probas conditionnelles pour tous les mots possibles du vocabulaire
        probasconds=[]
        for mot in vocabulaire:
            probasconds.append(probacond(mot," ".join(seqprec.split(' ')[-(n-1):])))
        #Selectionne le mot qui maximise la probabilité conditionnelle (greedy search)
        seqprec+=" "+vocabulaire[np.argmax(probasconds)]
    return seqprec

In [65]:
#Cas bigram
generergreedy(30,"<s>")

'<s> le plus de la première fois , le plus de la première fois , le plus de la première fois , le plus de la première fois , le plus'

In [73]:
#Cas trigram
generergreedy(50,"<s> il")

'<s> il est le plus souvent , les deux pays </s> <s> le premier ministre , daniel brugès , éditions du seuil , paris , éditions du seuil , paris , éditions du seuil , paris , éditions du seuil , paris , éditions du seuil , paris , éditions du seuil'

In [49]:
#Cas quatrigram
generergreedy(50,"<s> il est")

"<s> il est le premier à avoir utilisé le terme « aviation » , bien que le président de la république , élu d' eure-et-loir </s> <s> maurice leblanc a toujours gardé des relations étroites avec la russie , l' inde , la corée du sud , le brésil est le pays le"

In [57]:
#Cas 5-gram
generergreedy(50,"<s> il est le")

"<s> il est le fils de pépin le bref , la chancellerie ne se compose plus que d' ecclésiastiques et l' on peut encore traiter facilement : si l' on chiffre un message en utilisant la clef publique , alors on peut déchiffrer le message en utilisant la clef publique , alors on peut"

Remarque : ça se met souvent à boucler ! Pour éviter ça, on peut introduire une part d'aléatoire dans le choix du mot suivant, par exemple en tirant selon la distribution de probabilité plutôt qu'en choisissant systématiquement le plus probable.

Beam Search Method

In [123]:
#Génère une séquence de nbmots mots conditionnellement à la séquence précédente seqprec
#Avec la méthode Beam Search pour k meilleurs séquences conservées à chaque étape
def genererbeam(nbmots,seqprec,k):
    sequences=[[seqprec,1]]
    #A chaque etape
    for i in range(nbmots):
        #calcule tous les candidats possibles de l'étape (sequence totale et proba associée)
        candidates=[]
        for j in range(len(sequences)):
            seq, prob = sequences[j]
            for mot in vocabulaire:
                candidates.append([seq+" "+mot,prob*probacond(mot," ".join(seq.split(' ')[-(n-1):]))])
        # ordonne les candidats selon le score
        ordered = sorted(candidates, key=lambda tup:tup[1])
        # sélectionne les k meilleurs
        sequences = ordered[-k:]
    return sequences

In [104]:
#Cas bigram
genererbeam(30,"<s>",3)

[['<s> le premier ministre de la fin de la ville de la ville de la ville de la ville de la ville de la ville de la ville de la ville',
  7.486873168181089e-34],
 ['<s> le premier ministre de la fin de la ville de la ville de la ville de la ville de la ville de la ville de la ville de la première',
  8.241045373767097e-34],
 ['<s> le premier ministre de la fin de la ville de la ville de la ville de la ville de la ville de la ville de la première fois , le',
  9.530640308839064e-34]]

In [116]:
#Cas trigram
genererbeam(30,"<s> il",3)

[["<s> il s' agit d' un point de vue de la population </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> la première",
  2.592672590660988e-27],
 ["<s> il s' agit d' un point de vue de la population </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le pays",
  2.841209628240636e-27],
 ["<s> il s' agit d' un point de vue de la population </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le pays </s> <s> le premier",
  3.1430881512412038e-27]]

In [124]:
#Cas quatrigram
genererbeam(50,"<s> il est",3)

[["<s> il est également possible de porter la balle à deux mains </s> <s> lance : terme générique désignant une arme offensive dotée d' un fer emmanché sur une hampe </s> <s> jetés à terre , leurs corps sont foulés par les vainqueurs représentés sous des formes mixtes , en partie grâce aux",
  2.0868169263891186e-11],
 ["<s> il est également possible de porter la balle à deux mains </s> <s> lance : terme générique désignant une arme offensive dotée d' un fer emmanché sur une hampe </s> <s> jetés à terre , leurs corps sont foulés par les vainqueurs représentés sous des formes mixtes , en partie , par",
  2.608521157986398e-11],
 ["<s> il est également possible de porter la balle à deux mains </s> <s> lance : terme générique désignant une arme offensive dotée d' un fer emmanché sur une hampe </s> <s> jetés à terre , leurs corps sont foulés par les vainqueurs représentés sous des formes mixtes , en partie grâce à",
  8.347267705556474e-11]]

In [ ]:
#Cas 5-gram
genererbeam(30,"<s> il est le",3)

Améliorations :
-> lissage (smoothing)
-> Apprendre les fréquences de tous les k-grams pour k<n pour pouvoir switcher à un k plus petit si le k-gram recherché est absent lors de la prédiction
-> Travailler avec les log-probas pour être sûr de ne pas perdre en précision -> Le temps pour générer le texte me semble très long, il y a sans doute moyen d'optimiser le code